In [2]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf


NameError: name 'core' is not defined

In [ ]:
# Define drawing and landmark utility functions
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Initialize hand landmark detection with specific parameters
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands=2)

# Video capture setup
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # RGB to BGR conversion for MediaPipe
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (75, 75))
    # Hand landmark detection
    results = hands.process(image)

    # Draw hand landmarks and bounding box on the image
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for idx, landmark in enumerate(hand_landmarks.landmark):
                x = int(landmark.x * image.shape[1])
                y = int(landmark.y * image.shape[0])
                cv2.putText(image, str(idx), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the processed image with landmarks
    cv2.imshow('Test', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# When all the process is done
# Release the capture and destroy all windows
cap.release()
cv2.destroyAllWindows()

In [2]:
import cv2

image = cv2.imread('../datasets/Filtered Raw Images/Aku/annotated_000008.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (240, 240))
cv2.imshow('awda', image)
cv2.waitKey(0)  # Add this line to wait indefinitely for a key press
cv2.destroyAllWindows()  # Add this line to close all OpenCV windows after a key press


In [24]:
np.array([1,3,43,6,5]).reshape(1, -1)

array([[ 1,  3, 43,  6,  5]])

In [30]:
import joblib
import numpy as np
import pandas as pd

sample = pd.read_csv('../fix-data.csv')
sample = np.array(sample)
scaler = joblib.load("scaler.pkl")

for i in range(len(samples[:5])):
    current_sample = sample[i][1:-1] 
    current_sample = np.array(current_sample).reshape(1, -1)
    transformed_sample = scaler.transform(current_sample)
    sample[i][1:-1] = transformed_sample.flatten() 
    break
    # without scaler
    # samples[i] =  np.expand_dims(samples[i][1:], axis=0)
print(sample[:1])

[['A_0.jpg' -1.7815987986568862 0.5958334327417503 0.6353401246010253
  -1.6699358919913159 0.8230597988173596 -0.6783253341414758
  -1.3515750268349014 0.912997413369571 -0.9133481667728659
  -1.017809495811953 0.9646039992320188 -1.0925566078553028
  -0.674402759052964 0.9725940193365538 -1.2352716576600598
  -1.124710654667919 0.5834635521676054 -1.1535814548455496
  -0.6677210796979521 0.5594375833815637 -1.486347398524908
  -0.3102582627069685 0.528237838195568 -1.588895196517843
  -0.03732832173647564 0.4590032171029787 -1.6537175216829747
  -1.306796819329158 0.40429750269682785 -1.534713118425242
  -0.9261463360266025 0.6168771381218267 -1.934236681747126
  -1.1276139844683888 0.8610218267641183 -1.89248719792594
  -1.346145352623372 0.8987320038908418 -1.745859985669686
  -1.5201434098932014 0.2955069259077762 -1.8248551038201593
  -1.3170727710725814 0.6021925556938819 -2.1094006551408566
  -1.5145870373182155 0.7790785888717825 -1.8978080619239552
  -1.6577049298714954 0.745

C:\Users\ASUS TUF\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\ASUS TUF\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


# Testing Realtime Detection

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the SavedModel
model_path = 'Models/Handsign_detection'  # Path to the directory containing the saved_model.pb
model = tf.saved_model.load(model_path)

# Ensure all custom objects are registered (if any)
# from custom_objects import CustomLayer  # Example of importing custom layers

# Get the inference function and wrap it in a tf.function decorator
@tf.function
def infer(input_tensor):
    return model.signatures['serving_default'](input_tensor)

# Function to preprocess the frame for the model
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    frame = cv2.resize(frame, (224, 224))  # Resize to the input size expected by the model
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    frame = frame / 255.0  # Normalize pixel values
    frame = frame.astype(np.float32)  # Ensure the data type is float32
    return frame

# Function to draw bounding box and label on the frame
def draw_prediction(frame, boxes, classes):
    h, w, _ = frame.shape
    for box, cls in zip(boxes, classes):
        ymin, xmin, ymax, xmax = box
        (startX, startY, endX, endY) = (int(xmin * w), int(ymin * h), int(xmax * w), int(ymax * h))
        
        # Draw bounding box
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
        
        # Draw label
        label = f"Class: {cls.numpy()}"
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Capture video from the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    input_data = preprocess_frame(frame)
    input_tensor = tf.convert_to_tensor(input_data, dtype=tf.float32)
    
    try:
        # Perform inference
        outputs = infer(input_tensor)
        
        # Assuming the model's output contains bounding boxes and class predictions
        detection_boxes = outputs['detection_boxes'].numpy()[0]  # Adjust key as needed
        detection_classes = outputs['detection_classes'].numpy()[0]  # Adjust key as needed

        # Draw predictions on the frame
        draw_prediction(frame, detection_boxes, detection_classes)
        
        # Display the frame
        cv2.imshow('Hand Sign Detection', frame)
        
    except Exception as e:
        print(f"Inference error: {e}")
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Inference error: Graph execution error:

Detected at node 'model/landmarks_detector_v3/map/while/PyFunc' defined at (most recent call last):
Node: 'model/landmarks_detector_v3/map/while/PyFunc'
ValueError: Could not find callback with key=pyfunc_13 in the registry.
Traceback (most recent call last):

  File "D:\Apps\anaconda\envs\swara\lib\site-packages\tensorflow\python\ops\script_ops.py", line 256, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_13 in the registry.


	 [[{{node model/landmarks_detector_v3/map/while/PyFunc}}]] [Op:__inference_infer_788]
Inference error: Graph execution error:

Detected at node 'model/landmarks_detector_v3/map/while/PyFunc' defined at (most recent call last):
Node: 'model/landmarks_detector_v3/map/while/PyFunc'
ValueError: Could not find callback with key=pyfunc_13 in the registry.
Traceback (most recent call last):

  File "D:\Apps\anaconda\envs\swara\lib\site-pa